In [ ]:
import logging
import os
import requests
from tqdm import tqdm
import pandas as pd
import time

In [ ]:
# Read the CSV file
df = pd.read_csv('/Users/carboni/Downloads/download_gdrive.csv')  # Replace 'your_file.csv' with the actual path to your CSV file

# Assuming your CSV file has a column named 'google_drive_link' with the links to Google Drive PDFs
for link in df['Media URL']:
    # Extract the file ID from the Google Drive link
    file_id = link.split('/')[-2]

    # Construct the direct download link
    download_link = f'https://drive.google.com/uc?id={file_id}'

    # Define the file name (you can customize this based on your requirements)
    file_name = f'{file_id}.pdf'

    # Download the file
    gdown.download(download_link, output=file_name, quiet=False)
    print(f'Downloaded: {file_name}')

In [ ]:

# Read the CSV file
df = pd.read_csv('download_gdrive.csv')  # Replace 'your_file.csv' with the actual path to your CSV file

# Create a logger object
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

# Create a file handler
handler = logging.FileHandler('error.log')
handler.setLevel(logging.ERROR)

# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

# Define the path to the folder where you want to save the files
folder_path = 'pdfs'


downloaded_count = 0

for link in tqdm(df['Media URL']):
    # Stop after 50 items downloaded and pause for 60 seconds
    if downloaded_count == 50:
        print("Pausing for 60 seconds...")
        time.sleep(60)
        downloaded_count = 0

    # Extract the file ID from the Google Drive link
    file_id = link.split('/')[-2]

    # Construct the direct download link
    download_link = f'https://drive.google.com/uc?id={file_id}'

    # Define the file name (you can customize this based on your requirements)
    file_name = f'{file_id}.pdf'
    file_path = os.path.join(folder_path, file_name)

    # Check if the file already exists in the folder_path
    if os.path.exists(file_path):
        print(f'{file_name} already exists. Skipping...')
        continue

    try:
        # Download the file and save it to the specified folder
        response = requests.get(download_link, stream=True)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            total_size_in_bytes = int(response.headers.get('content-length', 0))
            block_size = 1024  # 1 Kibibyte
            progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
            with open(file_path, 'wb') as file:
                for data in response.iter_content(block_size):
                    progress_bar.update(len(data))
                    file.write(data)
            progress_bar.close()
            print(f'Downloaded: {file_name}')
            downloaded_count += 1  # Increment the counter
        else:
            # Log the error if the request was not successful
            logger.error(f'Error downloading {file_name}. Status code: {response.status_code}')
            
    except Exception as e:
        # Log the error
        logger.error(f'Error downloading {file_name}: {e}')


  5%|▌         | 6.82M/131M [00:20<01:05, 1.91MB/s]